# Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from scipy.stats import norm

# Setup

In [2]:
def prob_nd1_elo(elo1, elo2, r=0, var=8, t=1): # 8 looks best
    elo1+=100
    if elo2>elo1:
        temp=elo1
        elo1=elo2
        elo2=temp
        return 1-norm.cdf((np.log(elo1/elo2)+(r+var/2)*(t/365)) / ((var*t/365)**.5))
    return norm.cdf((np.log(elo1/elo2)+(r+var/2)*(t/365)) / ((var*t/365)**.5))

In [3]:
NBA_data= pd.read_csv("nbaallelo.csv")
NBA_data

,gameorder,game_id,lg_id,_iscopy,year_id,date_game,seasongame,is_playoffs,team_id,fran_id,...,win_equiv,opp_id,opp_fran,opp_pts,opp_elo_i,opp_elo_n,game_location,game_result,forecast,notes
0,1,194611010TRH,NBA,0,1947,11/1/1946,1,0,TRH,Huskies,...,40.294830,NYK,Knicks,68,1300.0000,1306.7233,H,L,0.640065,NaN
1,1,194611010TRH,NBA,1,1947,11/1/1946,1,0,NYK,Knicks,...,41.705170,TRH,Huskies,66,1300.0000,1293.2767,A,W,0.359935,NaN
2,2,194611020CHS,NBA,0,1947,11/2/1946,1,0,CHS,Stags,...,42.012257,NYK,Knicks,47,1306.7233,1297.0712,H,W,0.631101,NaN
3,2,194611020CHS,NBA,1,1947,11/2/1946,2,0,NYK,Knicks,...,40.692783,CHS,Stags,63,1300.0000,1309.6521,A,L,0.368899,NaN
4,3,194611020DTF,NBA,0,1947,11/2/1946,1,0,DTF,Falcons,...,38.864048,WSC,Capitols,50,1300.0000,1320.3811,H,L,0.640065,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126309,63155,201506110CLE,NBA,0,2015,6/11/2015,100,1,CLE,Cavaliers,...,60.309792,GSW,Warriors,103,1790.9591,1809.9791,H,L,0.546572,NaN
126310,63156,201506140GSW,NBA,0,2015,6/14/2015,102,1,GSW,Warriors,...,68.013329,CLE,Cavaliers,91,1704.3949,1700.7391,H,W,0.765565,NaN
126311,63156,201506140GSW,NBA,1,2015,6/14/2015,101,1,CLE,Cavaliers,...,60.010067,GSW,Warriors,104,1809.9791,1813.6349,A,L,0.234435,NaN
126312,63157,201506170CLE,NBA,0,2015,6/16/2015,102,1,CLE,Cavaliers,...,59.290245,GSW,Warriors,105,1813.6349,1822.2881,H,L,0.481450,NaN


In [4]:
NBA_data.columns

Index(['gameorder', 'game_id', 'lg_id', '_iscopy', 'year_id', 'date_game',
       'seasongame', 'is_playoffs', 'team_id', 'fran_id', 'pts', 'elo_i',
       'elo_n', 'win_equiv', 'opp_id', 'opp_fran', 'opp_pts', 'opp_elo_i',
       'opp_elo_n', 'game_location', 'game_result', 'forecast', 'notes'],
      dtype='object')

In [5]:
NBA_data['nd1_forecast'] = NBA_data.apply(lambda x: prob_nd1_elo(x['elo_i'], x['opp_elo_i']), axis=1)
NBA_data['results'] = (NBA_data['game_result'] == 'W')*1

C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  
C:\Users\rhnma\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


In [6]:
NBA_var=np.var(NBA_data['elo_i'])
NBA_mean=np.mean(NBA_data[NBA_data['team_id']=='NYK']['elo_i'])

print(NBA_mean, NBA_var/(126314**.5))

1497.6156040908302 35.382742825274356


In [7]:
NBA_data[['elo_i', 'opp_elo_i', 'forecast', 'nd1_forecast', 'results']]

,elo_i,opp_elo_i,forecast,nd1_forecast,results
0,1300.0000,1300.0000,0.640065,1.0,0
1,1300.0000,1300.0000,0.359935,1.0,1
2,1300.0000,1306.7233,0.631101,1.0,1
3,1306.7233,1300.0000,0.368899,1.0,0
4,1300.0000,1300.0000,0.640065,1.0,0
...,...,...,...,...,...
126309,1723.4149,1790.9591,0.546572,1.0,0
126310,1809.9791,1704.3949,0.765565,1.0,1
126311,1704.3949,1809.9791,0.234435,0.0,0
126312,1700.7391,1813.6349,0.481450,0.0,0


In [8]:
regdf1=NBA_data[['game_id','forecast', 'nd1_forecast']]
model1 = smf.ols(formula='nd1_forecast ~ forecast', data=regdf1).fit()
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           nd1_forecast   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                 7.571e+04
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        15:04:19   Log-Likelihood:                -43953.
No. Observations:              126314   AIC:                         8.791e+04
Df Residuals:                  126312   BIC:                         8.793e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1331      0.002     54.584      0.000       0.128       0.138
forecast       1.2325      0.004    275.163      0.000       1.224       1.241
==============================================================================
Omnibus:                     9676.888   Durbin-Watson:                   1.911
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5163.959
Skew:                          -0.340   Prob(JB):                         0.00
Kurtosis:                       2.279   Cond. No.                         5.85
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 538 Model Test Results

In [9]:
regdf2=NBA_data[['game_id','forecast', 'results']]
model2 = smf.ols(formula='forecast ~ results', data=regdf2).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               forecast   R-squared:                       0.185
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                 2.864e+04
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        15:04:19   Log-Likelihood:                 27686.
No. Observations:              126314   AIC:                        -5.537e+04
Df Residuals:                  126312   BIC:                        -5.535e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4075      0.001    526.892      0.000       0.406       0.409
results        0.1851      0.001    169.227      0.000       0.183       0.187
==============================================================================
Omnibus:                     6278.891   Durbin-Watson:                   3.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2485.259
Skew:                          -0.000   Prob(JB):                         0.00
Kurtosis:                       2.313   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
log_reg1 = sm.Logit(regdf2['results'], regdf2['forecast']).fit() 
log_reg1.summary()

Optimization terminated successfully.
         Current function value: 0.678589
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.02100
Time:                        15:04:20   Log-Likelihood:                -85715.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
forecast       0.6341      0.011     59.958      0.000       0.613       0.655
==============================================================================
"""

# ND1 Test Results

In [11]:
regdf3=NBA_data[['game_id','nd1_forecast', 'results']]
model3 = smf.ols(formula='nd1_forecast ~ results', data=regdf3).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           nd1_forecast   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     9195.
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        15:04:20   Log-Likelihood:                -69176.
No. Observations:              126314   AIC:                         1.384e+05
Df Residuals:                  126312   BIC:                         1.384e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6365      0.002    382.283      0.000       0.633       0.640
results        0.2258      0.002     95.893      0.000       0.221       0.230
==============================================================================
Omnibus:                    19414.326   Durbin-Watson:                   2.271
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23719.474
Skew:                          -1.018   Prob(JB):                         0.00
Kurtosis:                       2.402   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
log_reg2 = sm.Logit(regdf3['results'], regdf3['nd1_forecast']).fit() 
log_reg2.summary()

Optimization terminated successfully.
         Current function value: 0.684611
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.01232
Time:                        15:04:20   Log-Likelihood:                -86476.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
nd1_forecast     0.3036      0.007     46.172      0.000       0.291       0.317
================================================================================
"""

# Weighted Mix

In [13]:
a=.25
NBA_data['w_forecast']= NBA_data['nd1_forecast']*a+ NBA_data['forecast']*(1-a)

In [14]:
regdf4=NBA_data[['game_id','w_forecast', 'results']]
model4 = smf.ols(formula='w_forecast ~ results', data=regdf4).fit()
model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             w_forecast   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                 2.423e+04
Date:                Mon, 04 Jan 2021   Prob (F-statistic):               0.00
Time:                        15:04:20   Log-Likelihood:                 10368.
No. Observations:              126314   AIC:                        -2.073e+04
Df Residuals:                  126312   BIC:                        -2.071e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4647      0.001    523.939      0.000       0.463       0.466
results        0.1953      0.001    155.660      0.000       0.193       0.198
==============================================================================
Omnibus:                     8850.789   Durbin-Watson:                   2.840
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7200.243
Skew:                          -0.502   Prob(JB):                         0.00
Kurtosis:                       2.399   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
log_reg3 = sm.Logit(regdf3['results'], regdf4['w_forecast']).fit() 
log_reg3.summary()

Optimization terminated successfully.
         Current function value: 0.680376
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                results   No. Observations:               126314
Model:                          Logit   Df Residuals:                   126313
Method:                           MLE   Df Model:                            0
Date:                Mon, 04 Jan 2021   Pseudo R-squ.:                 0.01842
Time:                        15:04:20   Log-Likelihood:                -85941.
converged:                       True   LL-Null:                       -87554.
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
w_forecast     0.5265      0.009     56.278      0.000       0.508       0.545
==============================================================================
"""